In [5]:
import os
import subprocess



ImportError: cannot import name 'load_dataset' from 'datasets' (unknown location)

In [13]:
if not os.path.exists('datasets/diode.tar.gz'):
    subprocess.run(
        ['curl', '-L', '-o', 'datasets/diode.tar.gz', 'http://diode-dataset.s3.amazonaws.com/train.tar.gz'],
        check=True
    )

In [6]:
if not os.path.exists('datasets/nyu_depth_v2'):
    import tensorflow_datasets as tfds
    tfds.load(
        'nyu_depth_v2',
        data_dir='datasets'
    )